# Census Data Tools

MORPC works regularly with census data, including but not limited to ACS 5 and 1-year, Decennial Census, PEP, and geographies. The following module is useful for gathering and organizing census data for processes in various workflow. Those workflows are linked when appropriate. 

In [ ]:
import morpc

## API functions and variables

api_get() is a low-level wrapper for Census API requests that returns the results as a pandas dataframe. If necessary, it splits the request into several smaller requests to bypass the 50-variable limit imposed by the API.  

The resulting dataframe is indexed by GEOID (regardless of whether it was requested) and omits other fields that are not requested but which are returned automatically with each API request (e.g. "state", "county") 

In [ ]:
url = 'https://api.census.gov/data/2022/acs/acs1'
params = {
    "get": "GEO_ID,NAME,B01001_001E",
    "for": "county:049,041",
    "in": "state:39"
}

In [ ]:
api = morpc.census.api_get(url, params)

In [ ]:
api

## American Community Survey (ACS) Data Class

When using ACS data, generally we will be digesting data produded using the [morpc-censusacs-fetch](https://github.com/morpc/morpc-censusacs-fetch) workflow. The data that is produced from that script is by default saved in its output_data folders ./morpc-censusacs-fetch/output_data/

The Census ACS Fetch script leverages the `acs_data` class form `morpc.census`


### Create an initial object which represents a variable in the ACS data api.

The class takes 3 arguments:

1. variable group number
2. the year
3. the type of survey (1 or 5 year estimates)

In [ ]:
import morpc

In [ ]:
acs = morpc.census.acs_data('B02001', '2023', '5')

The initial call creates queries the Census for the variable definitions and returns a dictionary of the available variables in the group. see `acs.VARS`

In [ ]:
acs.VARS

The initial call alse fetchs a list of dimensions from a cached json file in ./morpc/census/acs_variable_group.json and is stored in morpc.census.ACS_VAR_GROUPS.

#### Manual verfication for variable dimension names. 

The list of dimensions are automatically created from the Census Variable labels and need verified before being used. If the dimesion names have not be verified, the will not be stored. Navigate to the JSON and check to make sure that there are the correct number of dimension and that they are in the correct order. Change the verfication field to `true`.

In [ ]:
acs.DIMENSIONS

### Query the API for the deisred variables and geography

The `.query()` method queries the API and caches the data in memory under `acs.DATA`. At the same time it creates a frictionless schema that corrosponds with the data. 

#### scope:
These are pre-defined sumlevels and scopes for commonly queried geographies. see `morpc.census.SCOPES`.

In [ ]:
morpc.census.SCOPES

In [ ]:
acs = acs.query(scope='region15-counties')

In [ ]:
data = acs.DATA
data.head()

### For custom queries, use for and in parameters to pass to api query. 

#### for_param:
(optional) The geographies for which to call the the query "state:*" represents all states. "state:39" represent Ohio.

#### in_param:
(optional) A filter for the for parameter. In combinations this allows you do call for small geograhpies inside larger ones. 

> Examples: for_param="county:\*", in_param="state:39" would get all counties in Ohio.
> for_param="tract:\*", in_param='state:39,county:041,049' gets all census tracts in Delaware and Franklin Counties.

### Filter the variables using the get parameter

#### get_param:
(Optional) If you want to return a subset of variables, they can be passed here as a list.

### Dimension Tables

When the query is called the class makes table with the dimensions included that can be used to get summaries of the data. 

This can be used to get quick queries for summaries. 

In [ ]:
acs = morpc.census.acs_data('B02001', '2023', '5').query(scope='region15-counties')

In [ ]:
acs.DIM_TABLE.LONG

In [ ]:
acs.DIM_TABLE.WIDE

In [ ]:
acs.DIM_TABLE.WIDE.loc(axis=0)[('Total', 'Black or African American alone'),'Total'].T

### Save raw data (not dim table) as a frictionless resource with schema

After querying the data, save the data as a frictionless resource with reasonable descriptors. 

In [ ]:
acs.save(output_dir='./temp_data/')

In [ ]:
acs.SCHEMA

In [ ]:
acs.RESOURCE

## Load data from cached file

In [1]:
import morpc

In [2]:
acs = morpc.census.acs_data('B02001', '2023', '5').load(scope='region15-counties', dirname='./temp_data/')

morpc.frictionless.load_data | INFO | Loading Frictionless Resource file at location morpc-acs5-2023-region15-counties-b02001.resource.yaml
morpc.frictionless.load_data | INFO | Using schema path specified in resource file.
morpc.frictionless.load_data | INFO | Loading data, resource file, and schema (if applicable) from their source locations
morpc.frictionless.load_data | INFO | --> Data file: morpc-acs5-2023-region15-counties-b02001.csv
morpc.frictionless.load_data | INFO | --> Resource file: morpc-acs5-2023-region15-counties-b02001.resource.yaml
morpc.frictionless.load_data | INFO | --> Schema file: morpc-acs5-2023-region15-counties-b02001.schema.yaml
morpc.frictionless.load_data | INFO | Loading data.


## Georeference the data to map

In [3]:
acs = acs.georeference()

morpc.frictionless.load_data | INFO | Loading Frictionless Resource file at location ../../morpc-geos-collect/output_data/morpc-geos.resource.yaml
morpc.frictionless.load_data | INFO | Ignoring schema as directed by useSchema parameter.
morpc.frictionless.load_data | INFO | Loading data, resource file, and schema (if applicable) from their source locations
morpc.frictionless.load_data | INFO | --> Data file: ../../morpc-geos-collect/output_data/morpc-geos.gpkg
morpc.frictionless.load_data | INFO | --> Resource file: ../../morpc-geos-collect/output_data/morpc-geos.resource.yaml
morpc.frictionless.load_data | INFO | --> Schema file: Not available. Ignoring schema.
morpc.frictionless.load_data | INFO | Loading data.
morpc.frictionless.load_data | INFO | Skipping casting of field types since we are ignoring schema.


/home/user/miniconda3/envs/morpc_env/lib/python3.11/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver GPKG does not support open option DRIVER


In [5]:
acs.DATA.explore(column='B02001_001E')

ImportError: The 'folium', 'matplotlib' and 'mapclassify' packages are required for 'explore()'. You can install them using 'conda install -c conda-forge folium matplotlib mapclassify' or 'pip install folium matplotlib mapclassify'.

## Below should still be functional, but hoping to implement into ACS class

#### Load the data using frictionless.load_data()

In [ ]:
data, resource, schema = morpc.frictionless.load_data('./temp_data/morpc-acs5-2023-state-B01001.resource.yaml', verbose=False)

#### Using ACS_ID_FIELDS to get the fields ids

In [ ]:
morpc.census.acs_generate_universe_table(data.set_index("GEO_ID"), "B01001_001")

#### Create a dimension table with the data and the dimension names

In [ ]:
dim_table = morpc.census.acs_generate_dimension_table(data.set_index("GEO_ID"), schema, idFields=idFields, dimensionNames=["Sex", "Age group"])

In [ ]:
dim_table.loc[dim_table['Variable type'] == 'Estimate'].head()

### Build ACS Variable Group JSON for Dimension names

In [ ]:
import requests
r = requests.get('https://api.census.gov/data/2023/acs/acs5/variables.json')
varjson = r.json()

In [ ]:
groups = {}
for variable in varjson['variables']:
    if variable not in ['for', 'in', 'ucgid', 'GEO_ID', 'AIANHH', 'AIHHTL', 'AIRES', 'ANRC']:
        group = varjson['variables'][variable]['group']
        if not group[-1].isalpha():
            if group not in groups:
                groups[group] = {}
                groups[group]['concept'] = varjson['variables'][variable]['concept'].replace('Year and Over','Year & Over').replace('Indian and Alaska','Indian & Alaska').replace('Hawaiian and Other','Hawaiian & Other')
                groups[group]['dimensions'] = ['TOTAL'] + varjson['variables'][variable]['concept'].replace(' by ',':').replace('Year and Over','Year & Over').replace('Indian and Alaska','Indian & Alaska').replace('Hawaiian and Other','Hawaiian & Other').replace(' and ',':').split(':')
                groups[group]['dimensions_verified'] = False
                variables = {}
                for variable in varjson['variables']:
                    if varjson['variables'][variable]['group'] == group:
                        variables[variable] = varjson['variables'][variable]['label']
                variables = {k: v for k, v in sorted(variables.items(), key=lambda item: item[0])}
                groups[group]['variables'] = variables

In [ ]:
groups = {k: v for k, v in sorted(groups.items(), key=lambda item: item[0])}

In [ ]:
import json
with open('../morpc/census/acs_variable_groups.json', 'w') as file:
    json.dump(groups, file, indent=3)